In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import torch.nn.init as init
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from torch.optim.lr_scheduler import MultiStepLR

import numpy as np
import time 
from tqdm import tqdm

from generator_utils import *
from discriminator import *
from dataset import *

In [2]:
class GRU_plain(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, has_input=True, has_output=False, output_size=None):
        super(GRU_plain, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.has_input = has_input
        self.has_output = has_output

        if has_input:
            self.input = nn.Linear(input_size, embedding_size)
            self.rnn = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers,
                              batch_first=True)
        else:
            self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        if has_output:
            self.output = nn.Sequential(
                nn.Linear(hidden_size, embedding_size),
                nn.ReLU(),
                nn.Linear(embedding_size, output_size)
            )

        self.relu = nn.ReLU()
        # initialize
        self.hidden = None  # need initialize before forward run

        for name, param in self.rnn.named_parameters():
            if 'bias' in name:
                nn.init.constant_(param, 0.25)
            elif 'weight' in name:
                nn.init.xavier_uniform_(param,gain=nn.init.calculate_gain('sigmoid'))
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight = init.xavier_uniform_(m.weight, gain=nn.init.calculate_gain('relu'))

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size)).to(choose_device())

    def forward(self, input_raw, pack=False, input_len=None):
        if self.has_input:
            input = self.input(input_raw)
            input = self.relu(input)
        else:
            input = input_raw
        if pack:
            input = pack_padded_sequence(input, input_len, batch_first=True)
        output_raw, self.hidden = self.rnn(input, self.hidden)
        if pack:
            output_raw = pad_packed_sequence(output_raw, batch_first=True)[0]
        if self.has_output:
            output_raw = self.output(output_raw)
        # return hidden state at each time step
        return output_raw

In [33]:
def decode_adj(adj_output):
    '''
    From GraphRNN codebase
        recover to adj from adj_output
        note: here adj_output have shape (n-1)*m
    '''
    max_prev_node = adj_output.shape[1]
    adj = torch.zeros((adj_output.shape[0], adj_output.shape[0]))
    print(adj.size())
    reverse_adj = torch.flip(adj_output, dims=(1,))
    for i in range(adj_output.shape[0]):
        input_start = max(0, i - max_prev_node + 1)
        input_end = i + 1
        output_start = max_prev_node + max(0, i - max_prev_node + 1) - (i + 1)
        output_end = max_prev_node
        # adj[i, input_start:input_end] = adj_output[i,::-1][output_start:output_end]
        # print(adj[i, input_start:input_end].size())
        # print(reverse_adj[i,output_start:output_end][:, 0].size())
        adj[i, input_start:input_end] = reverse_adj[i,output_start:output_end][:, 0]
    adj_full = torch.zeros((adj_output.shape[0]+1, adj_output.shape[0]+1))
    n = adj_full.shape[0]
    adj_full[1:n, 0:n-1] = torch.tril(adj, 0)
    adj_full = adj_full + adj_full.T

    return adj_full

In [34]:
class GraphRNN(nn.Module):
    def __init__(self, args, device=choose_device()) -> None:
        super().__init__()
        self.args = args
        self.device = device
        self.rnn = GRU_plain(input_size=self.args.max_prev_node, embedding_size=self.args.embedding_size_rnn,
                        hidden_size=self.args.hidden_size_rnn, num_layers=self.args.num_layers, has_input=True,
                        has_output=True, output_size=self.args.hidden_size_rnn_output).to(self.device)
        self.output = GRU_plain(input_size=1, embedding_size=self.args.embedding_size_rnn_output,
                            hidden_size=self.args.hidden_size_rnn_output, num_layers=self.args.num_layers, has_input=True,
                            has_output=True, output_size=1).to(self.device)

        # load data state
        if args.load:
            fname = args.model_save_path + args.fname + 'lstm_' + str(args.load_epoch) + '.dat'
            self.rnn.load_state_dict(torch.load(fname))
            fname = args.model_save_path + args.fname + 'output_' + str(args.load_epoch) + '.dat'
            self.output.load_state_dict(torch.load(fname))

            args.lr = 0.00001
            epoch = args.load_epoch
            print('model loaded!, lr: {}'.format(args.lr))
        else:
            epoch = 1

    # ====Call these in training loop====
    def init_optimizer(self, lr):
        """Initialize optimizers and schedular for both RNNs"""
        self.optimizer_rnn = optim.Adam(list(self.rnn.parameters()), lr=lr)
        self.optimizer_output = optim.Adam(list(self.output.parameters()), lr=lr)
        self.scheduler_rnn = MultiStepLR(self.optimizer_rnn, milestones=self.args.milestones)
        self.scheduler_output = MultiStepLR(self.optimizer_output, milestones=self.args.milestones)
        return self.optimizer_rnn, self.optimizer_output, self.scheduler_rnn, self.scheduler_output

    def clear_gradient_models(self):
        self.rnn.zero_grad()
        self.output.zero_grad()

    def train(self, flag):
        if flag:
            self.rnn.train(True)
            self.output.train(True)
        else:
            self.rnn.train(False)
            self.output.train(False)

    def clear_gradient_opts(self):
        self.optimizer_rnn.zero_grad()
        self.optimizer_output.zero_grad()

    def all_steps(self):
        self.optimizer_rnn.step()
        self.optimizer_output.step()
        self.scheduler_rnn.step()
        self.scheduler_output.step()

    def sort_data_per_epoch(self, X, Y, length):
        x_unsorted = X.float()
        y_unsorted = Y.float()
        y_len_unsorted = length
        y_len_max = max(y_len_unsorted)
        x_unsorted = x_unsorted[:, 0:y_len_max, :]
        y_unsorted = y_unsorted[:, 0:y_len_max, :]
        y_len,sort_index = torch.sort(y_len_unsorted,0,descending=True)
        y_len = y_len.numpy().tolist()
        x = torch.index_select(x_unsorted,0,sort_index)
        y = torch.index_select(y_unsorted,0,sort_index)
        y_reshape = pack_padded_sequence(y,y_len,batch_first=True).data
        idx = [i for i in range(y_reshape.size(0)-1, -1, -1)]
        idx = torch.LongTensor(idx)
        y_reshape = y_reshape.index_select(0, idx)
        y_reshape = y_reshape.view(y_reshape.size(0),y_reshape.size(1),1)
        output_x = torch.cat((torch.ones(y_reshape.size(0),1,1),y_reshape[:,0:-1,0:1]),dim=1) # x's shape is determined by y's shape
        output_y = y_reshape
        output_y_len = []
        output_y_len_bin = np.bincount(np.array(y_len))
        for i in range(len(output_y_len_bin)-1,0,-1):
            count_temp = np.sum(output_y_len_bin[i:]) # count how many y_len is above i
            output_y_len.extend([min(i,y.size(2))]*count_temp) # put them in output_y_len; max value should not exceed y.size(2)

        # pack into variable
        x = Variable(x).to(self.device)
        y = Variable(y).to(self.device)
        output_x = Variable(output_x).to(self.device)
        output_y = Variable(output_y).to(self.device)
        batch_size = x_unsorted.size(0)
        return x, y, output_x, output_y, y_len, output_y_len, batch_size
    
    # ======================================

    def forward(self, noise, X, Y, length):
        """
        X: noise/latent vector
        args: arguments dictionary
        test_batch_size: number of graphs you want to generate
        """
        # provide a option to change number of graphs generated
        output_batch_size = self.args.test_batch_size
        input_hidden = torch.stack(self.rnn.num_layers*[noise]).to(self.device)
        self.rnn.hidden = input_hidden # expected shape: (num_layer, batch_size, hidden_size)

        x, y, output_x, output_y, y_len, output_y_len, _ = self.sort_data_per_epoch(X, Y, length)
        
        h = self.rnn(x, pack=True, input_len=y_len)
        h = pack_padded_sequence(h,y_len,batch_first=True).data # get packed hidden vector
        # reverse h
        idx = [i for i in range(h.size(0) - 1, -1, -1)]
        idx = Variable(torch.LongTensor(idx)).cuda()
        h = h.index_select(0, idx)
        hidden_null = Variable(torch.zeros(self.rnn.num_layers-1, h.size(0), h.size(1))).cuda()
        self.output.hidden = torch.cat((h.view(1,h.size(0),h.size(1)),hidden_null),dim=0) # num_layers, batch_size, hidden_size
        y_pred = self.output(output_x, pack=True, input_len=output_y_len)
        y_pred = F.sigmoid(y_pred)
        # clean
        y_pred = pack_padded_sequence(y_pred, output_y_len, batch_first=True)
        y_pred = pad_packed_sequence(y_pred, batch_first=True)[0]

        out = decode_adj(y_pred)
        print(out.size())
        # # TODO: change this part to noise vector might need resizing
        # # y_pred_long = Variable(torch.zeros(output_batch_size, self.args.max_num_node, self.args.max_prev_node)).to(self.device) # discrete prediction
        # y_pred_long = torch.zeros(output_batch_size, self.args.max_num_node, self.args.max_prev_node).to(self.device) # discrete prediction
        # # x_step = X.to(self.device) # shape:(batch_size, 1, self.args.max_prev_node)

        # x_step = torch.ones(output_batch_size, 1, self.args.max_prev_node).to(self.device)

        # # iterative graph generation
        # for i in range(self.args.max_num_node):
        #     # for each node
        #     # 1. we use rnn to create new node embedding
        #     # 2. we use output to create new edges

        #     # (1)
        #     h = self.rnn(x_step)
        #     hidden_null = torch.zeros(self.args.num_layers - 1, h.size(0), h.size(2)).to(self.device)
        #     x_step = torch.zeros(output_batch_size, 1, self.args.max_prev_node).to(self.device)
        #     output_x_step = torch.ones(output_batch_size, 1, 1).to(self.device)
        #     # (2)
        #     # self.output.hidden = torch.cat((h.permute(1,0,2), hidden_null), dim=0).to(self.device)
        #     # for j in range(min(self.args.max_prev_node,i+1)):
        #     #     output_y_pred_step = self.output(output_x_step)
        #     #     # print(output_y_pred_step.requires_grad)
        #     #     output_x_step = sample_sigmoid(output_y_pred_step, sample=True, sample_time=1, device=self.device)
        #     #     x_step[:,:,j:j+1] = output_x_step
        #     #     # self.output.hidden = Variable(self.output.hidden.data).to(self.device)
        #     # y_pred_long[:, i:i + 1, :] = x_step
        #     self.rnn.hidden = Variable(self.rnn.hidden.data).to(self.device)

        # y_pred_long_data = y_pred_long.long()

        return out

        # print(1)
        init_adj_pred = decode_adj(y_pred_long_data[0].cpu())
        adj_pred_list = torch.zeros((output_batch_size, init_adj_pred.size(0), init_adj_pred.size(1)))
        for i in range(output_batch_size):
            # adj_pred = decode_adj(y_pred_long_data[i].cpu().numpy())
            # adj_pred_list = np.append(adj_pred_list, adj_pred)
            # adj_pred_list.append(adj_pred)
            adj_pred_list[i, :, :] = decode_adj(y_pred_long_data[i].cpu())

        # return torch.Tensor(np.array(adj_pred_list))
        return adj_pred_list

In [35]:
loss = nn.MSELoss()

In [44]:
def train(args, train_inverter=False, num_layers=4, clamp_lower=-0.1, clamp_upper=0.1, lr=1e-3, betas=1e-5, lamb=0.1, loss_func='MSE', device=choose_device()):
    # get the dataset
    lr = 1e-4
    train, labels = get_dataset_with_label(args.graph_type) # entire dataset as train
    train_dataset = Graph_sequence_sampler_pytorch(train, labels, args)
    train_loader = get_dataloader_labels(train_dataset, args)
    noise_dim = args.hidden_size_rnn
    # print('noise dimension is: ', noise_dim)

    # initialize noise, optimizer and loss
    netG = GraphRNN(args=args)
    netD = NetD(stat_input_dim=128, stat_hidden_dim=64, num_stat=2)
    hg = list(netG.parameters())[2].register_hook(lambda grad: print(f"NetG parameter Update with gradient {grad}"))
    loss = nn.MSELoss()

    # check model parameters
    # for param in netD.parameters():
    #     print(param.name, param.data, param.requires_grad)
    # for param in netG.parameters():
    #     print(param.name, param.data, param.requires_grad)

    optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=[betas for _ in range(2)])
    G_optimizer_rnn, G_optimizer_output, G_scheduler_rnn, G_scheduler_output = netG.init_optimizer(lr=0.1) # initialize optimizers


    noise = torch.randn(args.batch_size, noise_dim).to(device)

    gen_iterations = 0
    for e in range(args.epochs):
        # for now, treat the input as adj matrices
        for i, data in tqdm(enumerate(train_loader), desc=f"Training epoch#{e+1}", total=len(train_loader)):
            X = data['x']
            Y = data['y']
            adj_mat = data['adj_mat']
            Y_len = data['len']

            # zero grad
            optimizerD.zero_grad()
            G_optimizer_rnn.zero_grad()
            G_optimizer_output.zero_grad()

            # skip uneven batch
            if adj_mat.size(0) != args.batch_size:
                continue
            
            # ========== Train Generator ==================
            netD.train(False)
            netG.train(True)
            # netG.clear_gradient_models()
            G_optimizer_rnn.zero_grad()
            G_optimizer_output.zero_grad()
            # in case our last batch was the tail batch of the dataloader,
            # make sure we feed a full batch of noise
            # noisev = noise.normal_(0,1)
            noisev = torch.randn(args.batch_size, noise_dim)
            # print(f"noise size: {noisev.size()}")
            fake = netG(noisev, X, Y, Y_len)
            # fake_tensor = netD(fake)
            # output = Variable(loss(fake, torch.ones((32, 6200)).cuda()), requires_grad=True)
            # print(fake.size())
            target = torch.ones((2405, 2405)).to('cuda')
            output = loss(fake.cuda(), target)
            output.backward()
            print(output.grad)

            # errG = fake_tensor
            # errG.backward()
            G_optimizer_rnn.step()
            G_optimizer_output.step()
            # # for p in netG.parameters()[0]:
            # # netG.all_steps()
            # gen_iterations += 1

            print(f"errG for generator: {output.item()}")


In [40]:
args = Args()

In [43]:
train(args=args)

calculating max previous node, total iteration: 12000
iter 0 times
iter 2400 times
iter 4800 times
iter 7200 times
iter 9600 times
max previous node: 10


Training epoch#1:   0%|          | 0/35 [00:00<?, ?it/s]/home/jgeng/.conda/envs/dev180/lib/python3.9/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.Size([2404, 2404])
torch.Size([2405, 2405])
torch.Size([2405, 2405])


Training epoch#1:   0%|          | 0/35 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!